# Generate Water Body Polygons

**What does this notebook do?** 
This code follows the following workflow:
* Generate a list of netCDF files within a specified folder location
* Opens each netCDF file and:
    * Keep only pixels observed at least x times
    * Keep only pixels identified as wet at least x% of the time
        * Here the code can take in two wetness thresholds, to produce two initial temporary polygon files. 
    * Convert the raster data into polygons
* Append the polygon set to a temporary shapefile
* Remove artificial polygon borders created at tile boundaries by merging polygons that intersect across Albers Tile boundaries
* Filter the combined polygon dataset (note that this step happens after the merging of Albers tile boundary polygons to ensure that artifacts are not created by part of a polygon being filtered out, while the remainder of the polygon that sits on a separate tile is treated differently).
    * Filter the polygons based on size
    * Remove polygons that intersect with Australia's coastline
    * Remove erroneous 'water' polygons within high-rise CBD areas
    * Combine the two generated wetness thresholds (optional)
    * Optional filtering for proximity to major rivers (as identified by the Geofabric dataset)
* Save out the final polygon set to a shapefile

**Required inputs:**
* NetCDF files with WOfS outputs that will be used to define the persistent water body polygons
    * Variable name: `TileFolder`
    * This folder can be either a custom extraction of datacube-stats (as was done here), or you can choose to use the WOfS summary tiles for all of Australia (see [here for further information](#Tiles)).
* A coastline polygon to filter out polygons generated from ocean pixels.
    * Variable name: `LandSeaMaskFile`
    * Here we have generated a hightide coastline using the [Intertidal Extents Model (ITEM) v2](http://pid.geoscience.gov.au/dataset/ga/113842) dataset. See [here for more details](#coastline)
* Urban high rise polygon dataset
    * Variable name: `UrbanMaskFile`
    * WOfS has a known limitation, where deep shadows thrown by tall CBD buildings are misclassified as water. This means that our algorithm is defining 'water bodies' around these misclassified shadows in capital cities. [See here](GenerateWaterBodyPolygons.ipynb#Urban) for a discussion of how an urban mask is produced. 
    
**Optional inputs:**
* River line dataset for filtering out polygons comprised of river segments.
    * Variable name: `MajorRiversDataset`
    * The option to filter out major rivers is provided, and so this dataset is optional if `FilterOutRivers = False`.
    * Here we use the [Bureau of Meteorology's Geofabric v 3.0.5 Beta (Suface Hydrology Network)](ftp://ftp.bom.gov.au/anon/home/geofabric/), filtered to only keep features tagged as `major rivers`. 
    * There are some identified issues with this data layer that make the filtering using this data inconsistent (see [the discussion here](#rivers))
    * We therefore turn this off during the production of the water bodies shapefile. 

**Running this workflow:** 

*NOTE: The Australia-wide analysis is too large to complete within a Jupyter Notebook. It was instead run on raijin. See this individual cell blocks below for more details*

**Code history**
This notebook is an updated version of `FindLotsOfDamsUsingWOFLsInaLoop.ipynb`. This copy was created to maintain the prototype workflow, while updating the operational workflow for water body polygon production.

**Date:** August 2019

**Author:** Claire Krause

**Quality Check Performed, March 2019:** Imam Alam, Alex Vincent, Bex Dunn, Damien Ayers

In [1]:
%pylab notebook

import rasterio.features
from shapely.geometry import Polygon, shape, mapping
from shapely.ops import unary_union
import geopandas as gp
import fiona
from fiona.crs import from_epsg
import xarray as xr
import pandas as pd
import glob
import os.path
import math
import geohash as gh
import re

Populating the interactive namespace from numpy and matplotlib


## Set up the functions for this script

In [3]:
def Generate_list_of_albers_tiles(TileFolder = 'TileFolder', CustomData = True):
    '''
    Generate a list of Albers tiles to loop through for the water body analysis. This 
    function assumes that the list of tiles will be generated from a custom 
    datacube-stats run, and the file names will have the format
    
    */wofs_summary_8_-37_{date}.nc
    
    The tile number is expected in the 2nd and 3rd last positions when the string has been
    broken using `_`. If this is not the case, then this code will not work, and will throw an error. 
    
    Parameters
    ----------
    
    TileFolder : str
        This is the path to the folder of netCDF files for analysis. If this is not provided, or an
        incorrect path name is provided, the code will exit with an error.
    CustomData : boolean
        This is passed in from elsewhere in the notebook. If this is not entered, the default parameter is True.
    
    Returns
    -------
    CustomRegionAlbersTiles: list
        List of Albers tiles across the analysis region. 
        E.g. ['8_-32', '9_-32', '10_-32', '8_-33', '9_-33']
    
    '''  
    if os.path.exists(TileFolder) == False:
        print('** ERROR ** \n'
            'You need to specify a folder of files for running a custom region')
        return
    
    # Grab a list of all of the netCDF files in the tile folder
    TileFiles = glob.glob(f'{TileFolder}*.nc')
    
    CustomRegionAlbersTiles = set()
    for filePath in TileFiles:
        AlbersTiles = re.split('[_\.]', filePath)
        if CustomData:
            # Test that the albers tile numbers are actually where we expect them to be in the file name
            try: 
                int(AlbersTiles[-4])
                int(AlbersTiles[-3])
            except ValueError:
                print('** ERROR ** \n'
                    'The netCDF files are expected to have the file format "*/wofs_summary_8_-37_{date}.nc",\n'
                    'with the Albers tile numbers in the 2nd and 3rd last positions when separated on `_`. \n'
                    'Please fix the file names, or alter the `Generate_list_of_albers_tiles` function.')
                return

            # Now that we're happy that the file is reading the correct Albers tiles
            ThisTile = f'{AlbersTiles[-3]}_{AlbersTiles[-2]}'
        else:
            # Test that the albers tile numbers are actually where we expect them to be in the file name
            try: 
                int(AlbersTiles[-3])
                int(AlbersTiles[-2])
            except ValueError:
                print('** ERROR ** \n'
                    'The netCDF files are expected to have the file format "*/wofs_filtered_summary_8_-37.nc",\n'
                    'with the Albers tile numbers in the 2nd and 3rd last positions when separated on `_` and `.`. \n'
                    'Please fix the file names, or alter the `Generate_list_of_albers_tiles` function.')
                return

            # Now that we're happy that the file is reading the correct Albers tiles
            ThisTile = f'{AlbersTiles[-3]}_{AlbersTiles[-2]}'
        CustomRegionAlbersTiles.add(ThisTile)
    CustomRegionAlbersTiles = list(CustomRegionAlbersTiles)
    return CustomRegionAlbersTiles

def Generate_list_of_tile_datasets(ListofAlbersTiles, Year, TileFolder = 'TileFolder', CustomData = True):
    '''
    Generate a list of Albers tiles datasets to loop through for the water body analysis. Here, the 
    ListofAlbersTiles is used to generate a list of NetCDF files where the Albers coordinates have 
    been substituted into the naming file format.
    
    Parameters
    ----------
    
    CustomRegionAlbersTiles: list
        List of albers tiles to loop through
        E.g. ['8_-32', '9_-32', '10_-32', '8_-33', '9_-33']
    Year: int
        Year for the analysis. This will correspond to the netCDF files for analysis.
    TileFolder : str
        This is the path to the folder of netCDF files for analysis. If this is not provided, or an
        incorrect path name is provided, the code will exit with an error.
    CustomData : boolean
        This is passed from elsewhere in the notebook. If this parameter is not entered, the default value
        is True.
    
    Returns
    -------
    Alltilespaths: list
        List of file paths to files to be analysed.
    
    '''  
    
    if os.path.exists(TileFolder) == False:
        print('** ERROR ** \n'
            'You need to specify a folder of files for running a custom region')
        raise

    Alltilespaths = []
    
    if CustomData:
        for tile in ListofAlbersTiles:
            Tiles = glob.glob(f'{TileFolder}*_{tile}_{Year}0101.nc')
            Alltilespaths.append(Tiles[0]) # Assumes only one file will be returned
    else:
        for tile in ListofAlbersTiles:
            # Use glob to check that the file actually exists in the format we expect
            Tiles = glob.glob(f'{TileFolder}wofs_filtered_summary_{tile}.nc')
            # Check that assumption by seeing if the returned list is empty
            if not Tiles:
                Tiles = glob.glob(f'{TileFolder}WOFS_3577_{tile}_summary.nc')
            # Check that we actually have something now
            if not Tiles:
                print('** ERROR ** \n'
                    'An assumption in the file naming conventions has gone wrong somewhere.\n'
                    'We assume two file naming formats here: {TileFolder}wofs_filtered_summary_{tile}.nc, \n'
                    'and {TileFolder}WOFS_3577_{tile}_summary.nc. The files you have directed to don\'t meet \n'
                    'either assumption. Please fix the file names, or alter the `Generate_list_of_albers_tiles` function.')
            Alltilespaths.append(Tiles[0]) # Assumes only one file will be returned
 
    return Alltilespaths

def Filter_shapefile_by_intersection(gpdData, gpdFilter, filtertype = 'intersects', invertMask = True, 
                                     returnInverse = False):
    '''
    Filter out polygons that intersect with another polygon shapefile. 
    
    Parameters
    ----------
    
    gpdData: geopandas dataframe
        Polygon data that you wish to filter
    gpdFilter: geopandas dataframe
        Dataset you are using as a filter
    
    Optional
    --------
    filtertype: default = 'intersects'
        Options = ['intersects', 'contains', 'within']
    invertMask: boolean
        Default = 'True'. This determines whether you want areas that DO ( = 'False') or DON'T ( = 'True')
        intersect with the filter shapefile.
    returnInnverse: boolean
        Default = 'False'. If true, then return both parts of the intersection - those that intersect AND 
        those that don't as two dataframes.
    
    Returns
    -------
    gpdDataFiltered: geopandas dataframe
        Filtered polygon set, with polygons that intersect with gpdFilter removed.
    IntersectIndex: list of indices of gpdData that intersect with gpdFilter
    
    Optional
    --------
    if 'returnInverse = True'
    gpdDataFiltered, gpdDataInverse: two geopandas dataframes
        Filtered polygon set, with polygons that DON'T intersect with gpdFilter removed.
    '''    
    
    # Check that the coordinate reference systems of both dataframes are the same
    
    #assert gpdData.crs == gpdFilter.crs, 'Make sure the the coordinate reference systems of the two provided dataframes are the same'
    
    Intersections = gp.sjoin(gpdFilter, gpdData, how="inner", op=filtertype)
    
    # Find the index of all the polygons that intersect with the filter
    IntersectIndex = sorted(set(Intersections['index_right']))
    
    # Grab only the polygons NOT in the IntersectIndex
    # i.e. that don't intersect with a river
    if invertMask:
        gpdDataFiltered = gpdData.loc[~gpdData.index.isin(IntersectIndex)]
    else:
        gpdDataFiltered = gpdData.loc[gpdData.index.isin(IntersectIndex)]
    
    if returnInverse:
        # We need to use the indices from IntersectIndex to find the inverse dataset, so we
        # will just swap the '~'.
        
        if invertMask:
            gpdDataInverse = gpdData.loc[gpdData.index.isin(IntersectIndex)]
        else:
            gpdDataInverse = gpdData.loc[~gpdData.index.isin(IntersectIndex)]
            
        return gpdDataFiltered, IntersectIndex, gpdDataInverse
    else:    
        
        return gpdDataFiltered, IntersectIndex

## Set up all the parameters for the script

<a id='valid'></a>
### Filter results based on number of valid observations

The number of valid WOfS observations for each pixel varies depending on the frequency of clouds and cloud shadow, and the proximity to high slope and terrain shadow, and the seasonal change in solar angle. 

The `count_clear` parameter provides a count of the number of valid observations each pixel recorded over the analysis period. We can use this parameter to mask out pixels that were infrequently observed. If this mask is not applied, pixels that were observed only once could be included, if that observation was wet (i.e. a single wet observation means the calculation of the frequency statistic would be (1 wet observation) / (1 total observation) = 100% frequency of wet observations).

Here we set the minimum number of observations to be 4 per year. For our 32 year analysis, this means the minimum number of valid observations is 128.

In [4]:
MinimumValidObs = 128

<a id='wetnessThreshold'></a>
### How wet does a pixel need to be to be included?
The value/s set here will be the minimum amount of time (as a decimal between 0 and 1) that you want water to be detected before it is included in the analysis. 

E.g. If this was set to 0.10, any pixels that are wet *at least* 10% of the time will be included. If you don't want to use this filter, set this value to 0.

Following the exploration of an appropriate wetness threshold for our purposes ([see above](#wetness)), we choose to set two thresholds here. The code is set up to loop through both wetness thresholds, and to write out two temporary shapefiles. These two shapefiles with two separate thresholds are then used together to combine polygons from both thresholds later on in the workflow.

Polygons identified by the secondary threshold that intersect with the polygons generated by the primary threshold will be extracted, and included in the final polygon dataset. This means that the **location** of polygons is set by the primary threshold, but the **shape** of these polygons is set by the secondary threshold.

Threshold values need to be provided as a list of either one or two floating point numbers. If one number is provided, then this will be used to generate the initial polygon dataset. If two thresholds are entered, the **first number becomes the secondary threshold, and the second number becomes the primary threshold**. If more than two numbers are entered, the code will generate an error below. 

In [2]:
AtLeastThisWet = [0.05, 0.10]

<a id='size'></a>
### How big/small should the polygons be?
This filtering step can remove very small and/or very large polygons. The size listed here is in m2. A single pixel in Landsat data is 25 m X 25 m = 625 m2. 

**MinSize**

E.g. A minimum size of 6250 means that polygons need to be at least 10 pixels to be included. If you don't want to use this filter, set this value to 0.

**MaxSize**

E.g. A maximum size of 1 000 000 means that you only want to consider polygons less than 1 km2. If you don't want to use this filter, set this number to `math.inf`. *NOTE: if you are doing this analysis for all of Australia, very large polygons will be generated offshore, in the steps prior to filtering by the Australian coastline. For this reason, we have used a `MaxSize` = Area of Lake Frome * 2. This will remove the huge ocean polygons, but keep large inland waterbodies that we want to map.*

In [3]:
MinSize = 3120 # 5 pixels
MaxSize = 5000000000 # approx area of Lake Eyre

<a id='rivers'></a>
### Do you want to filter out polygons that intersect with major rivers?

**Note that for the Water Body Polygon dataset, we set this filter to False (`FilterOutRivers = False`). The code has been kept here in case it's of use for further filtering of the results later on.**

We use the [Bureau of Meteorology's Geofabric v 3.0.5 Beta (Suface Hydrology Network)](ftp://ftp.bom.gov.au/anon/home/geofabric/) to filter out polygons that intersect with major rivers. This is done to remove river segments from the polygon dataset. We use the `SH_Network AHGFNetworkStream any` layer within the `SH_Network_GDB_V2_1_1.zip` geodatabase, and filter the dataset to only keep rivers tagged as `major`. It is this filtered dataset we use here.

Note that we reproject this dataset to `epsg 3577`, Australian Albers coordinate reference system. If this is not correct for your analysis, you can change this in the cell below.

If you don't want to filter out polygons that intersect with rivers, set this parameter to `False`.

#### Note when using the Geofabric to filter out rivers

The option to filter out rivers was switched off for the production of our water bodies dataset. During testing, the Geofabric dataset was shown to lead to inconsistencies in what was removed, and what remained within the dataset. 

* The Geofabric continues the streamline through on-river dams, which means these polygons are filtered out. This may not be the desired result. 

![Stream and Dam intersection](OnRiverDam.JPG "An in-river dam that would be removed by the river filter, but may not be the desired result")

In [6]:
FilterOutRivers = False

In [7]:
# Where is this file located?
MajorRiversDataset = '/g/data/r78/cek156/ShapeFiles/Geofabric_v2_1_1/SH_Network_GDB_V2_1_1_Major_Filtered.shp'

# Read in the major rivers dataset (if you are using it)
if FilterOutRivers:
    MajorRivers = gp.GeoDataFrame.from_file(MajorRiversDataset) 
    MajorRivers = MajorRivers.to_crs({'init':'epsg:3577'})

<a id='Tiles'></a>
### Set up the input datasets for the analysis

In [8]:
AllOfAustraliaAllTime = True

CustomData = False
AutoGenerateTileList = False

**All of Australia analysis**

If you would like to perform the analysis for all of Australia, using the published WOfS all time summaries, set `AllofAustraliaAllTime = True`. 

The WOfS all time summaries NetCDF files used are located in `/g/data/fk4/datacube/002/WOfS/WOfS_Stats_25_2_1/netcdf/`. These files contain the following three variables: `count_wet`, `count_clear` and `frequency`.

**Custom Data option**

If `CustomData = True`, you will need to specify the location of the data you would like to use for this analysis, setting `TileFolder` below, under the `if CustomData` code section below.  

If `CustomData = False`, the code will automatically look at the published WOfS all time summaries.

**Autogeneration of tile list**

`AutoGenerateTileList` will only be used if `AllOfAustraliaAllTime = False`. We only want to generate a list of tiles to iterate through if it will be a subset of a ll of the available data.

If you would like to automatically generate a list of tiles using the outputs of an analysis (e.g. we have previously run a custom `datacube-stats` analysis using this region, and so we can generate a list of tiles that we know covers this area using the outputs of this analysis), set `AutoGenerateTileList = True` and update the location of the output file directory.

If you would like to manually feed in a list of albers tiles, set `AutoGenerateTileList = False`, and feed in a list of tiles in the format:

```
ListofAlbersTiles = ['7_-34', '10_-40', '16_-34']
```
<br/>    
**NOTE** For testing and debugging, set `CustomData = True` and `AutoGenerateTileList = False`, then enter a list of tiles to run using the `ListofAlbersTiles` described above.

In [9]:
if CustomData:
    TileFolder = '/g/data/r78/cek156/datacube_stats/WOFSDamsAllTimeNSWMDB/'
else:
    TileFolder = '/g/data/fk4/datacube/002/WOfS/WOfS_Stats_25_2_1/netcdf/'

In [10]:
# We only want to generate the tile list if we are not doing all of Australia.
if not AllOfAustraliaAllTime:
    if AutoGenerateTileList:
        ListofAlbersTiles = Generate_list_of_albers_tiles(TileFolder, CustomData)
    else:
        ListofAlbersTiles = ['8_-32','19_-40', '10_-30', '14_-31', '13_-31', '18_-38', '19_-33', '9_-31', '20_-34', 
                         '6_-39', '17_-31', '12_-40', '13_-39', '10_-38', '6_-41', '16_-32', '9_-40', '10_-31', 
                         '16_-37', '10_-39', '16_-35', '7_-38', '10_-37', '17_-39', '9_-34', '19_-35', '15_-33', 
                         '15_-30', '11_-32', '20_-39', '17_-30', '13_-40', '7_-41', '17_-29', '12_-33', '15_-28',
                         '16_-40', '6_-35', '17_-40', '13_-38', '17_-42', '14_-39', '13_-29', '17_-37', '16_-38',
                         '9_-32', '16_-34', '9_-41', '11_-31', '7_-36', '16_-39', '18_-30', '9_-37', '20_-37',
                         '13_-32', '7_-40', '18_-41', '20_-32', '8_-41', '14_-28', '18_-39', '14_-43', '12_-39',
                         '20_-36', '8_-34', '17_-41', '12_-41', '18_-31', '11_-38', '18_-34', '14_-35', '12_-42',
                         '19_-39', '12_-34', '10_-42', '11_-35', '17_-35', '15_-41', '18_-33', '6_-37', '13_-41',
                         '10_-40', '14_-33', '13_-37', '8_-36', '6_-36', '16_-43', '18_-36', '14_-40', '15_-43',
                         '12_-30', '5_-39', '8_-39', '18_-35', '15_-39', '15_-29', '7_-34', '11_-34', '14_-41',
                         '15_-42', '16_-29', '16_-28', '14_-37', '8_-33', '6_-38', '19_-38', '13_-33', '16_-36',
                         '15_-37', '12_-38', '7_-35', '18_-40', '12_-31', '16_-41', '14_-38', '19_-37', '10_-34',
                         '14_-32', '12_-32', '14_-42', '15_-35', '16_-31', '19_-36', '7_-37', '11_-41', '14_-36',
                         '13_-35', '16_-42', '13_-36', '6_-40', '17_-36', '10_-41', '18_-37', '14_-29', '14_-30',
                         '20_-38', '17_-38', '12_-36', '10_-35', '9_-42', '21_-33', '12_-37', '17_-32', '15_-31',
                         '10_-36', '15_-36', '19_-34', '17_-34', '12_-35', '20_-40', '20_-33', '19_-31', '20_-35',
                         '16_-30', '18_-32', '12_-29', '11_-30', '15_-32', '16_-33', '8_-37', '10_-33', '13_-34', 
                         '11_-33', '13_-30', '11_-36', '8_-40', '11_-40', '10_-32', '9_-38', '11_-42', '11_-39',
                         '15_-34', '21_-34', '13_-42', '9_-35', '8_-42', '17_-33', '8_-38', '13_-28', '15_-38',
                         '9_-36', '8_-35', '11_-37', '14_-34', '15_-40', '9_-33', '19_-32', '9_-39', '7_-39']

<a id='coastline'></a>
### Read in a land/sea mask

A high tide coastline for Australia was used to mask out polygons that are continuously connected to the ocean, or an estuary. This coastline was generated using the [Intertidal Extents Model (ITEM) v2](http://pid.geoscience.gov.au/dataset/ga/113842) dataset.

For a detailed description of how this coastline was created, see [this notebook](./CreateAustralianCoastlineUsingITEM.ipynb)

In [3]:
LandSeaMaskFile = '/g/data/r78/cek156/ShapeFiles/ITEMv2Coastline/ITEM_Ocean_Polygon.shp'

Coastline = gp.read_file(LandSeaMaskFile)
Coastline = Coastline.to_crs({'init': 'epsg:3577'})

<a id='Urban'></a>

### Read in a mask for high-rise CBDs

WOfS has a known limitation, where deep shadows thrown by tall CBD buildings are misclassified as water. This means that our algorithm is defining 'water bodies' around these misclassified shadows in capital cities. 

To address this problem, we use the [Australian Bureau of Statistics Statistical Area 3 shapefile (2016)](http://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.001July%202016?OpenDocument#Data) to define a spatial footprint for Australia's CBD areas. 

We use the following polygons as our CBD filter:

|SA3_CODE1|SA3_NAME16|
|---------|----------|
|11703    |Sydney Inner City|
|20604    |Melbourne City|
|30501    |Brisbane Inner|
|30901    |Broadbeach - Burleigh|
|30910    |Surfers Paradise|
|40101    |Adelaide City|
|50302    |Perth City|

In [14]:
UrbanMaskFile = '/g/data/r78/cek156/ShapeFiles/ABS_1270055001_sa3_2016_aust_shape/HighRiseCBD_ABS_sa3.shp'

CBDs = gp.read_file(UrbanMaskFile)
CBDs = CBDs.to_crs({'init': 'epsg:3577'})

## Generate the first temporary polygon dataset

This code section:

1. Checks that the `AtLeastThisWet` threshold has been correctly entered above
2. Sets up a `for` loop that allows the user to input multiple temporal datasets (see below)
3. Generates a list of netCDF files to loop through
4. Sets up a `for` loop for that list of files. Here we have separate data for each Landsat tile, so this loop loops through the list of tile files
5. Opens the netCDF `frequency` data and removes the `time` dimension (which in this case is only of size 1)
6. Opens the netCDF `count_clear` data and removes the `time` dimension (which in this case is only of size 1)
7. Removes any pixels not observed at least [`MinimumValidObs` times](#valid)
8. Sets up a `for` loop for the entered [`AtLeastThisWet` thresholds](#wetnessThreshold)
9. Masks out any data that does not meet the wetness threshold
10. Converts the data to a Boolean array, with included pixels == 1
11. Converts the raster array to a polygon dataset
12. Cleans up the polygon dataset
13. Resets the `geometry` to a shapely geometry
14. Merges any overlapping polygons
15. Convert the output of the merging back into a geopandas dataframe
16. Calculates the area of each polygon
17. Saves the results to a shapefile

Within this section you need to set up:
- **WaterBodiesShp:** The name and filepath of the intermediate output polygon set
- **WOFSshpMerged:** The filepath for the location of temp files during the code run
- **WOFSshpFiltered:** The name and filepath of the outputs following the [filtering steps](#filtering)
- **FinalName:** The name and file path of the final, completed waterbodies shapefile
- **years to analyse:** `for year in range(x,y)` - note that the last year is NOT included in the analysis. This for loop is set up to allow you to loop through multiple datasets to create multiple polygon outputs. If you only have one input dataset, set this to `range(<year of the analysis>, <year of the analysis + 1>)`

In [4]:
## Set up some file names for the inputs and outputs
# The name and filepath of the intermediate output polygon set
WaterBodiesShp = f'/g/data/r78/cek156/dea-notebooks/WaterbodyAreaMappingandMonitoring/AusAllTime01-005HybridWaterbodies/Temp'

# The name and filepath of the temp, filtered output polygon set
WOFSshpMerged = f'/g/data/r78/cek156/dea-notebooks/WaterbodyAreaMappingandMonitoring/AusAllTime01-005HybridWaterbodies/'
WOFSshpFiltered = '/g/data/r78/cek156/dea-notebooks/WaterbodyAreaMappingandMonitoring/AusAllTime01-005HybridWaterbodies/AusWaterBodiesFiltered.shp'

# Final shapefile name
FinalName = '/g/data/r78/cek156/dea-notebooks/WaterbodyAreaMappingandMonitoring/AusAllTime01-005HybridWaterbodies/AusWaterBodies.shp'

In [ ]:
# First, test whether the wetness threshold has been correctly set
if len(AtLeastThisWet) == 2:
    print(f'We will be running a hybrid wetness threshold. Please ensure that the major threshold is \n'
          f'listed second, with the supplementary threshold entered first.' 
          f'**You have set {AtLeastThisWet[-1]} as the primary threshold,** \n'
          f'**with {AtLeastThisWet[0]} set as the supplementary threshold.**')
elif len(AtLeastThisWet) == 1:
    print(f'You have not set up the hybrid threshold option. If you meant to use this option, please \n'
         f'set this option by including two wetness thresholds in the `AtLeastThisWet` variable above. \n'
         f'The wetness threshold we will use is {AtLeastThisWet}.')
else:
    raise ValueError (f'There is something wrong with your entered wetness threshold. Please enter a list \n'
                      f'of either one or two numbers. You have entered {AtLeastThisWet}. \n'
                      f'See above for more information')

# Now perform the analysis to generate the first iteration of polygons
for year in range(1980,1981):
    
    ### Get the list of netcdf file names to loop through
    if AllOfAustraliaAllTime:
        # Grab everything from the published WOfS all time summaries
        Alltiles = glob.glob(f'{TileFolder}*.nc')
    else:
        Alltiles = Generate_list_of_tile_datasets(ListofAlbersTiles, year, TileFolder, CustomData)            
        
    for WOFSfile in Alltiles: 
        try:
            # Read in the data
            # Note that the netCDF files we are using here contain a variable called 'frequency',
            # which is what we are using to define our water polygons.
            # If you use a different netCDF input source, you may need to change this variable name here
            WOFSnetCDFData = xr.open_rasterio(f'NETCDF:{WOFSfile}:frequency')
            # Remove the superfluous time dimension
            WOFSnetCDFData = WOFSnetCDFData.squeeze()
            
            # Open the clear count variable to generate the minimum observation mask
            WOFSvalidcount = xr.open_rasterio(f'NETCDF:{WOFSfile}:count_clear')
            WOFSvalidcount = WOFSvalidcount.squeeze()

            # Filter our WOfS classified data layer to remove noise
            # Remove any pixels not abserved at least MinimumValidObs times
            WOFSValidFiltered = WOFSvalidcount >= MinimumValidObs
            
            for Thresholds in AtLeastThisWet:
                # Remove any pixels that are wet < AtLeastThisWet% of the time
                WOFSfiltered = WOFSnetCDFData > Thresholds

                # Now find pixels that meet both the MinimumValidObs and AtLeastThisWet criteria
                # Change all zeros to NaN to create a nan/1 mask layer
                # Pixels == 1 now represent our water bodies
                WOFSfiltered = WOFSfiltered.where((WOFSfiltered !=0) & (WOFSValidFiltered !=0))

                # Convert the raster to polygons
                # We use a mask of '1' to only generate polygons around values of '1' (not NaNs)
                WOFSpolygons = rasterio.features.shapes(WOFSfiltered.data.astype('float32'), 
                                                        mask = WOFSfiltered.data.astype('float32') == 1,
                                                        transform = WOFSnetCDFData.transform)
                # The rasterio.features.shapes returns a tuple. We only want to keep the geometry portion,
                # not the value of each polygon (which here is just 1 for everything)
                WOFSbreaktuple = (a for a, b in WOFSpolygons)

                # Put our polygons into a geopandas geodataframe
                PolygonGP = gp.GeoDataFrame(list(WOFSbreaktuple))

                # Grab the geometries and convert into a shapely geometry
                # so we can quickly calcuate the area of each polygon
                PolygonGP['geometry'] = None
                for ix, poly in PolygonGP.iterrows():
                    poly['geometry'] = shape(poly)

                # Set the geometry of the dataframe to be the shapely geometry we just created    
                PolygonGP = PolygonGP.set_geometry('geometry')
                # We need to add the crs back onto the dataframe
                PolygonGP.crs = {'init': 'epsg:3577'}

                # Combine any overlapping polygons
                MergedPolygonsGeoms = unary_union(PolygonGP['geometry'])

                # Turn the combined multipolygon back into a geodataframe
                MergedPolygonsGPD = gp.GeoDataFrame([poly for poly in MergedPolygonsGeoms])
                # Rename the geometry column
                MergedPolygonsGPD.columns = ['geometry']
                # We need to add the crs back onto the dataframe
                MergedPolygonsGPD.crs = {'init': 'epsg:3577'}

                # Calculate the area of each polygon again now that overlapping polygons
                # have been merged
                MergedPolygonsGPD['area'] = MergedPolygonsGPD['geometry'].area

                # Save the polygons to a shapefile
                schema = {'geometry': 'Polygon','properties': {'area': 'float'}}
                
                # Generate our dynamic filename
                FileName = f'{WaterBodiesShp}_{Thresholds}.shp'
                # Append the file name to the list so we can call it later on

                if os.path.isfile(FileName):
                    with fiona.open(FileName, "a", crs = from_epsg(3577), 
                                    driver = 'ESRI Shapefile', schema = schema) as output:
                        for ix, poly in MergedPolygonsGPD.iterrows():
                                        output.write(({'properties': {'area': poly['area']},
                                                       'geometry': mapping(shape(poly['geometry']))})) 
                else:
                    with fiona.open(FileName, "w", crs = from_epsg(3577), 
                                    driver = 'ESRI Shapefile', schema = schema) as output:
                        for ix, poly in MergedPolygonsGPD.iterrows():
                            output.write(({'properties': {'area': poly['area']},
                                           'geometry': mapping(shape(poly['geometry']))}))

        except:
            print(f'{WOFSfile} did not run. \n'
            f'This is probably because there are no waterbodies present in this tile.')

<a id='MergeTiles'></a>

## Merge polygons that have an edge at a tile boundary

Now that we have all of the polygons across our whole region of interest, we need to check for artifacts in the data caused by tile boundaries. 

We have created a shapefile that consists of the albers tile boundaries, plus a 1 pixel (25 m) buffer. This shapefile will help us to find any polygons that have a boundary at the edge of an albers tile. We can then find where polygons touch across this boundary, and join them up.

Within this section you need to set up:
- **AlbersBuffer:** The file location of a shapefile that is a 1 pixel buffer around the Albers tile boundaries

*NOTE: for the Australia-wide analysis, the number and size of polygons means that this cell cannot be run in this notebook. Instead, we ran this cell on raijin*

```
#!/bin/bash
#PBS -P r78
#PBS -q hugemem
#PBS -l walltime=96:00:00
#PBS -l mem=500GB
#PBS -l jobfs=200GB
#PBS -l ncpus=7
#PBS -l wd
#PBS -lother=gdata1a
 
module use /g/data/v10/public/modules/modulefiles/
module load dea

PYTHONPATH=$PYTHONPATH:/g/data/r78/cek156/dea-notebooks

```

In [ ]:
AlbersBuffer = gp.read_file(
    '/g/data/r78/cek156/ShapeFiles/AlbersBuffer25m.shp')

for Threshold in AtLeastThisWet:
    print(f'Working on {Threshold} shapefile')
    # We are using the more severe wetness threshold as the main polygon dataset.
    # Note that this assumes that the thresholds have been correctly entered into the 'AtLeastThisWet'
    # variable, with the higher threshold listed second.
    WaterPolygons = gp.read_file(f'{WaterBodiesShp}_{Threshold}.shp')

    # Find where the albers polygon overlaps with our dam polygons
    BoundaryMergedDams, IntersectIndexes, NotBoundaryDams= Filter_shapefile_by_intersection(WaterPolygons, AlbersBuffer, 
                                                                          invertMask=False, 
                                                                          returnInverse=True)

    # Now combine overlapping polygons in `BoundaryDams`
    UnionBoundaryDams = BoundaryMergedDams.unary_union

    # `Explode` the multipolygon back out into individual polygons
    UnionGDF = gp.GeoDataFrame(crs=WaterPolygons.crs, geometry=[UnionBoundaryDams])
    MergedDams = UnionGDF.explode()

    # Then combine our new merged polygons with the `NotBoundaryDams`
    # Combine New merged polygons with the remaining polygons that are not near the tile boundary
    AllTogether = gp.GeoDataFrame(pd.concat([NotBoundaryDams, MergedDams],
                                            ignore_index=True, sort=True)).set_geometry('geometry')

    # Calculate the area of each polygon
    AllTogether['area'] = AllTogether.area
    
    schema = {'geometry': 'Polygon','properties': {'area': 'float'}}

    print(f'Writing out {Threshold} shapefile')
    
    with fiona.open(f'{WOFSshpMerged}Union_{Threshold}.shp', "w", crs = from_epsg(3577), 
                    driver = 'ESRI Shapefile', schema = schema) as output:
        for ix, poly in AllTogether.iterrows():
            output.write(({'properties': {'area': poly['area']},
                           'geometry': mapping(shape(poly['geometry']))}))

<a id='Filtering'></a>

## Filter the merged polygons by:
- **Area:**
Based on the `MinSize` and `MaxSize` parameters set [here](#size).
- **Coastline:**
Using the `Coastline` dataset loaded [here](#coastline).
- **CBD location:**
Using the `CBDs` dataset loaded [here](#Urban).
- **Wetness thresholds:**
Here we apply the hybrid threshold described [here](#wetness)
- **Intersection with rivers (optional):**
Using the `MajorRivers` dataset loaded [here](#rivers)

*NOTE: for the Australia-wide analysis, the number and size of polygons means that this cell cannot be run in this notebook. Instead, we ran this cell on raijin*

```
#!/bin/bash
#PBS -P r78
#PBS -q hugemem
#PBS -l walltime=96:00:00
#PBS -l mem=500GB
#PBS -l jobfs=200GB
#PBS -l ncpus=7
#PBS -l wd
#PBS -lother=gdata1a
 
module use /g/data/v10/public/modules/modulefiles/
module load dea

PYTHONPATH=$PYTHONPATH:/g/data/r78/cek156/dea-notebooks

```

In [ ]:
AllTogether = gp.read_file(f'{WOFSshpMerged}Temp_{AtLeastThisWet[1]}.shp')
AllTogether['area'] = pd.to_numeric(AllTogether.area)

# Filter out any polygons smaller than MinSize, and greater than MaxSize
WaterBodiesBig = AllTogether.loc[((AllTogether['area'] > MinSize) & (AllTogether['area'] <= MaxSize))]

# Filter out any ocean in the pixel
WaterBodiesLand, IntersectIndexes = Filter_shapefile_by_intersection(WaterBodiesBig, Coastline, 
                                                   invertMask = True)

# WOfS has a known bug where deep shadows from high-rise CBD buildings are misclassified
# as water. We will use the ABS sa3 dataset to filter out Brisbane, Gold Coast, Sydney, 
# Melbourne, Adelaide and Perth CBDs. 
NotCities, IntersectIndexes = Filter_shapefile_by_intersection(WaterBodiesLand, CBDs)

# Check for hybrid wetness thresholds
if len(AtLeastThisWet) == 2:
    # Note that this assumes that the thresholds have been correctly entered into the 'AtLeastThisWet'
    # variable, with the supplementary threshold listed first.
    LowerThreshold = gp.read_file(f'{WOFSshpMerged}Union_{AtLeastThisWet[0]}.shp')
    LowerThreshold['area'] = pd.to_numeric(LowerThreshold.area)
    # Filter out those pesky huge polygons
    LowerThreshold = LowerThreshold.loc[(LowerThreshold['area'] <= MaxSize)]
    # Find where the albers polygon overlaps with our dam polygons
    BoundaryMergedDams, IntersectIndexes = Filter_shapefile_by_intersection(LowerThreshold,
                                                                            NotCities)
    # Pull out the polygons from the supplementary shapefile that intersect with the primary shapefile
    LowerThresholdToUse = LowerThreshold.loc[LowerThreshold.index.isin(IntersectIndexes)]
    # Concat the two polygon sets together
    CombinedPolygons = gp.GeoDataFrame(pd.concat([LowerThresholdToUse, NotCities], ignore_index=True))
    # Merge overlapping polygons
    CombinedPolygonsUnion = CombinedPolygons.unary_union
    # `Explode` the multipolygon back out into individual polygons
    UnionGDF = gp.GeoDataFrame(crs=LowerThreshold.crs, geometry=[CombinedPolygonsUnion])
    HybridDams = UnionGDF.explode()
else:
    print('You have not set up the hybrid threshold option. If you meant to use this option, please \n'
          'set this option by including two wetness thresholds in the `AtLeastThisWet` variable above')
    HybridDams = NotCities

# Here is where we do the river filtering (if FilterOutRivers == True)
if FilterOutRivers:
    WaterBodiesBigRiverFiltered, IntersectIndexes = Filter_shapefile_by_intersection(HybridDams, MajorRivers)
else:
    # If river filtering is turned off, then we just keep all the same polygons
    WaterBodiesBigRiverFiltered = HybridDams
    
# We need to add the crs back onto the dataframe
WaterBodiesBigRiverFiltered.crs = {'init': 'epsg:3577'}

# Calculate the area and perimeter of each polygon again now that overlapping polygons
# have been merged
WaterBodiesBigRiverFiltered['area'] = WaterBodiesBigRiverFiltered['geometry'].area
WaterBodiesBigRiverFiltered['perimeter'] = WaterBodiesBigRiverFiltered['geometry'].length

# Calculate the Polsby-Popper value (see below), and write out too
WaterBodiesBigRiverFiltered['PPtest'] = ((WaterBodiesBigRiverFiltered['area'] * 4 * math.pi) / 
                                         (WaterBodiesBigRiverFiltered['perimeter'] ** 2))

# Save the polygons to a shapefile
schema = {'geometry': 'Polygon','properties': {'area': 'float', 'perimeter':'float', 'PPtest':'float'}}

with fiona.open(WOFSshpFiltered, "w", crs = from_epsg(3577), 
                driver = 'ESRI Shapefile', schema = schema) as output:
    for ix, poly in WaterBodiesBigRiverFiltered.iterrows():
        output.write(({'properties': {'area': poly['area'], 
                                      'perimeter': poly['perimeter'], 
                                      'PPtest': poly['PPtest']},
                       'geometry': mapping(shape(poly['geometry']))}))


## Dividing up very large polygons

The size of polygons is determined by the contiguity of waterbody pixels through the landscape. This can result in very large polygons, e.g. where rivers are wide and unobscured by trees, or where waterbodies are connected to rivers or neighbouring waterbodies. The image below shows this for the Menindee Lakes, NSW. The relatively flat terrain in this part of Australia means that the 0.05 wetness threshold results in the connection of a large stretch of river and the individual lakes into a single large polygon that spans 154 km. This polygon is too large to provide useful insights into the changing water surface area of the Menindee Lakes, and needs to be broken into smaller, more useful polygons.

![Menindee Lakes original polygon](menindeeLakes.JPG)

We do this by applying the [Polsby-Popper test (1991)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2936284). The Polsby-Popper test is an assessment of the 'compactness' of a polygon. This method was originally developed to test the shape of congressional and state legislative districts, to prevent gerrymandering. 

The Polsby-Popper test examines the ratio between the area of a polygon, and the area of a circle equal to the perimeter of that polygon. The result falls between 0 and 1, with values closer to 1 being assessed as more compact.

\begin{align*}
PPtest = \frac{polygon\ area * 4\pi}{polygon\ perimeter^2}
\end{align*}


The Menindee Lakes polygon above has a PPtest value $\approx$ 0.00. 

We selected all polygons with a `PPtest` value <=0.005. This resulted in a subset of 186 polygons. 

![Polygons with a Polsby-Popper test score of less than 0.005](PPtestlessthan005.JPG)

The 186 polygons were buffered with a -50 meter (2 pixel) buffer to separate the polygons where they are connected bu two pixels or less. This allows us to split up these very large polygons by using natural thinning points. The resulting negatively buffered polygons was run through the `multipart to singlepart` tool in QGIS, to give the now separated polygons unique IDs. 

These polygons were then buffered with a +50 meter buffer to return the polygons to approximately their original size. These final polygons were used to separate the 186 original polygons identified above.

The process for dividing up the identified very large polygons varied depending on the polygon in question. Where large waterbodies (like the Menindee Lakes) were connected, the buffered polygons were used to determine the cut points in the original polygons. Where additional breaks were required, the [Bureau of Meteorology's Geofabric v 3.0.5 Beta (Suface Hydrology Network)](ftp://ftp.bom.gov.au/anon/home/geofabric/) `waterbodies` dataset was used as an additional source of information for breaking up connected segments.

The buffering method didn't work on large segments of river, which became a series of disconnected pieces when negatively and positively buffered. Instead, we used a combination of tributaries and man-made features such as bridges and weirs to segment these river sections.

## Final checks and recalculation of attributes

In [11]:
WaterBodiesBigRiverFiltered = gp.read_file(WOFSshpFiltered)

In [12]:
# Recalculate the area and perimeter of each polygon again following the manual checking
# step performed above
WaterBodiesBigRiverFiltered['area'] = WaterBodiesBigRiverFiltered['geometry'].area
WaterBodiesBigRiverFiltered['perimeter'] = WaterBodiesBigRiverFiltered['geometry'].length

In [13]:
# Remove the PPtest column, since we don't really want this as an attribute of the final shapefile
WaterBodiesBigRiverFiltered.drop(labels = 'PPtest', axis=1, inplace=True)

In [14]:
# Reapply the size filtering, just to check that all of the split and filtered waterbodies are
# still in the size range we want
DoubleCheckArea = WaterBodiesBigRiverFiltered.loc[((WaterBodiesBigRiverFiltered['area'] > MinSize) & 
                                                   (WaterBodiesBigRiverFiltered['area'] <= MaxSize))]

## Generate a unique ID for each polygon

A unique identifier is required for every polygon to allow it to be referenced. The naming convention for generating unique IDs here is the [geohash](geohash.org).

A Geohash is a geocoding system used to generate short unique identifiers based on latitude/longitude coordinates. It is a short combination of letters and numbers, with the length of the string a function of the precision of the location. The methods for generating a geohash are outlined [here - yes, the official documentation is a wikipedia article](https://en.wikipedia.org/wiki/Geohash).

Here we use the python package `python-geohash` to generate a geohash unique identifier for each polygon. We use `precision = 9` geohash characters, which represents an on the ground accuracy of <20 metres. This ensures that the precision is high enough to differentiate between waterbodies located next to each other.

In [15]:
# We need to convert from Albers coordinates to lat/lon, in order to generate the geohash
GetUniqueID = DoubleCheckArea.to_crs(epsg=4326)

# Generate a geohash for the centroid of each polygon
GetUniqueID['UID'] = GetUniqueID.apply(lambda x: gh.encode(x.geometry.centroid.y, x.geometry.centroid.x, 
                                                           precision=9), axis=1)

# Check that our unique ID is in fact unique
assert GetUniqueID['UID'].is_unique

# Make an arbitrary numerical ID for each polygon. We will first sort the dataframe by geohash
# so that polygons close to each other are numbered similarly
SortedData = GetUniqueID.sort_values(by = ['UID']).reset_index() 
SortedData['FID'] = SortedData.index

In [30]:
# The step above creates an 'index' column, which we don't actually want, so drop it.
SortedData.drop(labels = 'index', axis=1, inplace=True)

## Write out the final results to a shapefile

In [32]:
BackToAlbers = SortedData.to_crs(epsg=3577)
BackToAlbers.to_file(FinalName, driver='ESRI Shapefile')